# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [3]:
%load_ext autoreload
%autoreload 2

import sys, os, re, time #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct
from TelechargementDonnees import (JourneeSiteNba, PasDeMatchError, JoueursSiteNba, 
                                   DriverFirefox, gererCookieNba, gererCookieTtfl, 
                                   simplifierNomJoueur, Blessures, Calendrier)
import pandas as pd
import numpy as np
import time
from collections import Counter
from datetime import date, datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [2]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2021-03-30', dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
    #exporter
    #journee.saveCsv()
except PasDeMatchError as e : 
    print(e)

0 https://www.nba.com/game/cha-vs-was-0022000718/box-score#box-score
1 https://www.nba.com/game/phi-vs-den-0022000719/box-score#box-score
2 https://www.nba.com/game/atl-vs-phx-0022000171/box-score#box-score
3 https://www.nba.com/game/orl-vs-lac-0022000720/box-score#box-score


In [ ]:
# pour un ensemble de journee : 
for j in ['2021-01-%02d' %n  for n in range(25,27)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j,dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

## 2. Telecharger les données liees aux joueurs depuis le site internet 
> Dans le cas ou on souhaite telecharger la totalite des donnees de la page des joueuers
1. on se connecte à la page principale des joueurs
1. on affiche l'integralite des donnees

In [62]:
#creation de la classe
pagejoueuer=EnsembleJoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')

## 3. Creation calendrier

In [4]:
cal=Calendrier(1,'2021-04-13',2)

In [5]:
cal.telechargerCalendrier()

In [6]:
cal.calendrier

,equipe_exterieure,equipe_domicile,date_match,id_saison
0,BKN,MIN,2021-04-13,1
1,LAC,IND,2021-04-13,1
2,ATL,TOR,2021-04-13,1
3,LAL,CHA,2021-04-13,1
4,OKC,UTA,2021-04-13,1
5,MIA,PHX,2021-04-13,1
6,BOS,POR,2021-04-13,1
0,MIL,MIN,2021-04-14,1
1,CLE,CHA,2021-04-14,1
2,BKN,PHI,2021-04-14,1


# TESTS

In [14]:
with ct.ConnexionBdd('basket', 'boulot') as c:
    resultDateMatchRecent=c.sqlAlchemyConn.execute('SELECT max(date_match) FROM donnees_source.match')
    resultDateCalendrierRecent=c.sqlAlchemyConn.execute('SELECT max(date_match), max(id_saison) FROM donnees_source.calendrier')
    r=resultDateCalendrierRecent.fetchall()[0]

In [15]:
r[0]

datetime.date(2021, 4, 12)